### Introduction and Schema Diagram

In this project, we will be working with a modified version of the [Chinook](https://github.com/lerocha/chinook-database) database to answer business questions using SQLite. 

The Chinook database contains information about a fictional digital music shop - kind of like a mini-iTunes store. It contains information about the artists, songs, and albums from the music shop, as well as information on the shop's employees, customers, and the customers purchases. This information is contained in eleven tables. Here's a schema diagram for the Chinook database:

<img src="chinook-schema.svg" />

### Connect to database

In [7]:
%%capture
%load_ext sql
%sql sqlite:///chinook.db

'Connected: None@chinook.db'

### List all tables

In [6]:
%%sql
SELECT
    name,
    type
FROM sqlite_master
WHERE type IN ("table","view");

Done.


name,type
album,table
artist,table
customer,table
employee,table
genre,table
invoice,table
invoice_line,table
media_type,table
playlist,table
playlist_track,table


### Top 10 genres in USA

Lets find the top 10 genres that sell the most tracks in USA.

In [7]:
%%sql
WITH usa_tracks_sold AS (
    SELECT il.* FROM invoice_line il
    INNER JOIN invoice i on il.invoice_id = i.invoice_id
    INNER JOIN customer c on i.customer_id = c.customer_id
    WHERE c.country = "USA"
)
SELECT
    g.name AS genre,
    count(uts.invoice_line_id) AS tracks_sold
FROM usa_tracks_sold uts
INNER JOIN track t on t.track_id = uts.track_id
INNER JOIN genre g on g.genre_id = t.genre_id
GROUP BY genre
ORDER BY tracks_sold DESC
LIMIT 10;

Done.


genre,tracks_sold
Rock,561
Alternative & Punk,130
Metal,124
R&B/Soul,53
Blues,36
Alternative,35
Latin,22
Pop,22
Hip Hop/Rap,20
Jazz,14


Based on the sales, we can see that the top 3 genres sold in the USA are Rock, Alternative & Punk and Metal.

### Employee Sales Performance

Each customer for the Chinook store gets assigned to a sales support agent within the company when they first make a purchase. Lets find the total sales assigned to each sales support agent.

In [7]:
%%sql

WITH customer_support_rep_sales AS
    (
     SELECT
         i.customer_id,
         c.support_rep_id,
         SUM(i.total) AS total
     FROM invoice i
     INNER JOIN customer c ON i.customer_id = c.customer_id
     GROUP BY 1,2
    )

SELECT
    e.first_name || " " || e.last_name AS employee,
    SUM(csrs.total) AS total_sales
FROM customer_support_rep_sales csrs
INNER JOIN employee e ON e.employee_id = csrs.support_rep_id
GROUP BY employee;

Done.


employee,total_sales
Jane Peacock,1731.5099999999998
Margaret Park,1584.0000000000002
Steve Johnson,1393.92


Jane Peacock has the highest number of sales.

### Sales by Country

Next, we will analyze the sales data by country.

In [6]:
%%sql
SELECT c.country, COUNT(DISTINCT c.customer_id) AS num_of_customers, SUM(i.total) AS total_sales
FROM customer c 
JOIN invoice i ON c.customer_id = i.customer_id 
GROUP BY 1 
ORDER BY 2 DESC 
LIMIT 5;

Done.


country,num_of_customers,total_sales
USA,13,1040.4899999999998
Canada,8,535.5900000000001
Brazil,5,427.68000000000006
France,5,389.0699999999999
Germany,4,334.62


The highest sales are from USA with a total of 13 customers.

### Top 10 Albums

Lastly, lets see the top 5 albums, as calculated by the number of times a track from that album has been purchased.

In [8]:
%%sql
SELECT ta.title album, ta.artist_name artist, COUNT(*) tracks_purchased 
FROM invoice_line il
INNER JOIN (SELECT t.track_id, artist.name artist_name, album.title 
            FROM track t
            INNER JOIN album ON album.album_id = t.album_id 
            INNER JOIN artist ON artist.artist_id = album.artist_id
            ) ta 
            ON ta.track_id = il.track_id 
GROUP BY album 
ORDER BY tracks_purchased DESC LIMIT 5;

Done.


album,artist,tracks_purchased
Are You Experienced?,Jimi Hendrix,187
Faceless,Godsmack,96
Mezmerize,System Of A Down,93
Get Born,JET,90
The Doors,The Doors,83


Looks like tracks from "Are You Experienced?" by Jimi Hendrix are the most popular.